In [1]:
%matplotlib inline

# Numerical libraries
import numpy as np   

# Import Linear Regression machine learning library
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso

from sklearn.metrics import r2_score

# to handle data in form of rows and columns 
import pandas as pd    

# importing ploting libraries
import matplotlib.pyplot as plt   

#importing seaborn for statistical plots
import seaborn as sns

In [2]:
mpg_df = pd.read_csv("auto-mpg.csv")  

In [3]:
mpg_df.head()

,mpg,cyl,disp,hp,wt,acc,yr,origin,car name
0,18.0,8,307.0,130,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165,3693,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150,3436,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150,3433,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140,3449,10.5,70,1,ford torino


In [4]:
mpg_df = mpg_df.drop('car name', axis=1)
mpg_df['origin'] = mpg_df['origin'].replace({1: 'america', 2: 'europe', 3: 'asia'})
mpg_df = pd.get_dummies(mpg_df, columns=['origin'])
mpg_df = mpg_df.replace('?', np.nan)
mpg_df = mpg_df.apply(lambda x: x.fillna(x.median()),axis=0)

# separate independent and dependent variables

In [5]:
# Copy all the predictor variables into X dataframe. Since 'mpg' is dependent variable drop it
X = mpg_df.drop('mpg', axis=1)

# Copy the 'mpg' column alone into the y dataframe. This is the dependent variable
y = mpg_df[['mpg']]


In [6]:
from sklearn import preprocessing

# scale all the columns of the mpg_df. This will produce a numpy array
X_scaled = preprocessing.scale(X)
X_scaled = pd.DataFrame(X_scaled, columns=X.columns)  # ideally the training and test should be 

y_scaled = preprocessing.scale(y)
y_scaled = pd.DataFrame(y_scaled, columns=y.columns)  # ideally the training and test should be 

In [7]:

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.30, random_state=1)

# fit a simple linear model

In [8]:
regression_model = LinearRegression()
regression_model.fit(X_train, y_train)

for idx, col_name in enumerate(X_train.columns):
    print("The coefficient for {} is {}".format(col_name, regression_model.coef_[0][idx]))

The coefficient for cyl is -0.0859226425444871
The coefficient for disp is 0.386150176689544
The coefficient for hp is -0.10637514644618848
The coefficient for wt is -0.7965737428612104
The coefficient for acc is 0.021846813318919682
The coefficient for yr is 0.3959410531014956
The coefficient for origin_america is -0.09399896644893752
The coefficient for origin_asia is 0.04491789013804927
The coefficient for origin_europe is 0.07243059852959186


In [9]:
intercept = regression_model.intercept_[0]

print("The intercept for our model is {}".format(intercept))

The intercept for our model is 0.015510225561902412


In [10]:
mpg_df.head()

,mpg,cyl,disp,hp,wt,acc,yr,origin_america,origin_asia,origin_europe
0,18.0,8,307.0,130,3504,12.0,70,True,False,False
1,15.0,8,350.0,165,3693,11.5,70,True,False,False
2,18.0,8,318.0,150,3436,11.0,70,True,False,False
3,16.0,8,304.0,150,3433,12.0,70,True,False,False
4,17.0,8,302.0,140,3449,10.5,70,True,False,False


# Create a regularized RIDGE model and note the coefficients

In [11]:
ridge = Ridge(alpha=.3)
ridge.fit(X_train,y_train)
print ("Ridge model:", (ridge.coef_))
    

Ridge model: [[-0.0800581   0.36661042 -0.10890119 -0.78324655  0.01917898  0.39442138
  -0.0930884   0.04466769  0.07153523]]


# Create a regularized LASSO model and note the coefficients

In [12]:
lasso = Lasso(alpha=0.1)
lasso.fit(X_train,y_train)
print ("Lasso model:", (lasso.coef_))

# Observe, many of the coefficients have become 0 indicating drop of those dimensions from the model

Lasso model: [-0.         -0.         -0.01464723 -0.60711757  0.          0.29460087
 -0.04017427  0.          0.        ]


## Let us compare their scores

In [13]:
print(regression_model.score(X_train, y_train))
print(regression_model.score(X_test, y_test))


0.8141025501610559
0.8433135132808831


In [14]:
print(ridge.score(X_train, y_train))
print(ridge.score(X_test, y_test))

0.8140828080856514
0.8437999817350273


In [15]:
print(lasso.score(X_train, y_train))
print(lasso.score(X_test, y_test))

0.7878910251573477
0.8315130533007058


In [16]:
# More or less similar results but with less complex models.  Complexity is a function of variables and coefficients
## Note - with Lasso, we get equally good result in test though not so in training.  Further, the number of dimensions is much less
# in LASSO model than ridge or un-regularized model

# Let us generate polynomial models reflecting the non-linear interaction between some dimensions

In [17]:
from sklearn.preprocessing import PolynomialFeatures

In [18]:
poly = PolynomialFeatures(degree = 2, interaction_only=True)

#poly = PolynomialFeatures(2)

In [19]:
X_poly = poly.fit_transform(X_scaled)
X_train, X_test, y_train, y_test = train_test_split(X_poly, y, test_size=0.30, random_state=1)
X_train.shape

(278, 46)

# Fit a simple non regularized linear model on poly features-

In [20]:
regression_model.fit(X_train, y_train)
print(regression_model.coef_[0])


[-3.68062164e-13 -9.15885875e-02 -4.60930574e-01 -1.94228137e+00
 -5.26751657e+00 -6.17386508e-01  3.06970256e+00  3.28354286e+10
 -2.97555893e+12  2.75778783e+12 -1.82690401e+00  1.67677241e-01
  2.12425426e+00  1.51874766e+00 -1.58460012e+00  3.81258742e+12
  3.14221495e+12  2.99925192e+12  1.96068557e-01  1.58020548e+00
 -1.12360543e+00  2.73170990e+00  8.44268674e+11  6.95819755e+11
  6.64161672e+11 -5.73392590e-01 -2.21927084e-01 -1.57244795e+00
  3.25942363e+11  2.68631470e+11  2.56409400e+11 -1.20684130e-01
  2.37562211e-01 -1.35582985e+12 -1.11743243e+12 -1.06659201e+12
  5.48001244e-01 -3.52414742e+11 -2.90449175e+11 -2.77234454e+11
 -4.01729306e+10 -3.31092691e+10 -3.16028790e+10 -2.56292310e+11
  4.19757859e+12 -2.55047084e+12]


In [21]:
ridge = Ridge(alpha=.3)
ridge.fit(X_train,y_train)
print ("Ridge model:", (ridge.coef_))

Ridge model: [[ 0.          0.06740642 -0.61900803 -1.97236759 -5.15141317 -0.62282102
   3.04381568  0.1723188   0.15891088 -0.38553368 -1.4895438   0.02925116
   1.72762625  1.4201127  -1.38679985 -0.05712906  1.13569653 -1.11720961
   0.30089657  1.53987731 -0.84218996  2.38658282  0.21457492  0.50684735
  -0.80377008 -0.47592772 -0.30069342 -1.50318104 -0.61710306  0.43440955
   0.32933252 -0.14480549  0.25597746  0.47585604 -0.93880283  0.37865359
   0.4784103  -0.67039722  0.22915648  0.61211567 -0.49887694  0.4033774
   0.21155723 -0.29118156  0.41878367  0.1184465 ]]


In [22]:
print(ridge.score(X_train, y_train))
print(ridge.score(X_test, y_test))


0.9025975935207239
0.8673792928418461


In [23]:
lasso = Lasso(alpha=0.01)
lasso.fit(X_train,y_train)
print ("Lasso model:", (lasso.coef_))


Lasso model: [ 0.         -0.         -0.08692269 -1.94971176 -5.29180738 -0.47273225
  2.98385949 -0.          0.         -0.         -0.79527606 -0.05882027
  1.1561088   1.08164446 -0.9493974  -0.          1.23352833 -0.87530628
 -0.          1.38994992 -0.4329787   1.86995071 -0.          0.
 -0.         -0.         -0.24115467 -1.24997101 -0.6285489   0.
  0.         -0.16019703  0.          0.26564551 -0.61312578 -0.
  0.47635543 -0.89394     0.          0.3945189  -0.68253648  0.12935612
  0.         -0.23102258  0.38217499  0.        ]


In [24]:
print(lasso.score(X_train, y_train))
print(lasso.score(X_test, y_test))


0.9013410674767774
0.8704180363604553
